In [1]:
import pandas as pd
import os 
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

In [2]:
# 定义数据源文件夹的路径
data_folder = "../可视化"

# 创建一个空的DataFrame用于存储所有的数据
all_data = pd.DataFrame()

# 遍历数据源文件夹中的每一个Excel文件
for file_name in os.listdir(data_folder):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(data_folder, file_name)
        
        try:
            # 读取Excel文件
            data = pd.read_excel(file_path)
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            continue

        # 在数据中添加新的列，列名为'Channel_code'，值为当前文件名（不包含扩展名）
        data['industry'] = os.path.splitext(file_name)[0]
        
        # 将读取的数据追加到总的DataFrame中
        all_data = pd.concat([all_data, data], ignore_index=True)

In [3]:
all_data.head()

,Name,URL,Category,Phone,Address,Website,yellow pages city,yellow pages state,store qty type,industry
0,Applause Dancewear AL,https://www.yellowpages.com/birmingham-al/mip/...,"Shoe Stores, Dancing Supplies",(205) 791-7209,"1629 Oxmoor Rd, Birmingham, AL 35209",https://www.applausedancewear.net/,Birmingham,AL,1,Shoes Stores
1,Rack Room Shoes,https://www.yellowpages.com/birmingham-al/mip/...,"Shoe Stores, Sportswear",(205) 655-4243,"1636 Gadsden Hwy Ste 100, Birmingham, AL 35235",https://www.rackroomshoes.com/store/381,Birmingham,AL,100+,Shoes Stores
2,Foot Locker,https://www.yellowpages.com/birmingham-al/mip/...,"Shoe Stores, Sportswear",(205) 655-5886,"5006 Pinnacle Sq, Birmingham, AL 35235",http://www.footlocker-inc.com/,Birmingham,AL,100+,Shoes Stores
3,Finish Line,https://www.yellowpages.com/birmingham-al/mip/...,"Shoe Stores, Sportswear",(205) 655-3687,"5024 Pinnacle Sq Ste 108, Birmingham, AL 35235",Website not found,Birmingham,AL,100+,Shoes Stores
4,The Shoe Outlet,https://www.yellowpages.com/birmingham-al/mip/...,"Shoe Stores, Outlet Stores",(205) 942-6001,"430 Green Springs Hwy Ste 17, Birmingham, AL 3...",Website not found,Birmingham,AL,1,Shoes Stores


In [4]:
industry_list = all_data['industry'].unique()

In [5]:
industry_list

array(['Shoes Stores', 'Clothing Stores', 'Vape Shops', 'Toy Shops',
       'Health Food', 'Gift Shops'], dtype=object)

In [6]:
df1 = all_data.dropna(subset=['store qty type'])

In [7]:
type_list = df1['store qty type'].unique()
type_list

array([1, '100+', '51~100', '2~5', '21~30', '16~20', '6~10', '41~50',
       '11~15', '31~40'], dtype=object)

In [8]:
# 按行业、店铺数量类型、州和城市进行分组，并聚合计算商家数量和店铺数量
df1 = df1.groupby(['industry','store qty type','yellow pages state','yellow pages city']).agg(
    merchant_qty = ('Name',pd.Series.nunique),
    store_qty = ('Address','count')
).reset_index()

In [9]:
df1

,industry,store qty type,yellow pages state,yellow pages city,merchant_qty,store_qty
0,Clothing Stores,1,AK,Anchorage,64,64
1,Clothing Stores,1,AK,Eagle River,4,4
2,Clothing Stores,1,AK,Wasilla,1,1
3,Clothing Stores,1,AL,Alabaster,7,7
4,Clothing Stores,1,AL,Athens,1,1
...,...,...,...,...,...,...
44491,Vape Shops,6~10,WI,Mount Pleasant,1,1
44492,Vape Shops,6~10,WI,Waukesha,1,1
44493,Vape Shops,6~10,WI,West Allis,1,1
44494,Vape Shops,6~10,WV,Charleston,1,1


In [10]:
# 将 'store qty type' 字段转换为文本格式
df1['store qty type'] = df1['store qty type'].astype(str)


In [11]:
city_info = pd.read_excel(r'../数据源/city.xlsx')
city_info.head()

,yellow pages state,yellow pages city,location,latitude,longitude
0,AK,Anchorage,"Anchorage, Anchorage, USA",61.216313,-149.894852
1,AK,Eagle River,"Eagle River, Eagle River, USA",47.407605,-88.295644
2,AK,Wasilla,"Wasilla, Wasilla, USA",61.580516,-149.440540
3,AL,Alabaster,"Alabaster, Alabaster, USA",33.225299,-86.803720
4,AL,Athens,"Athens, Athens, USA",39.328924,-82.101248


In [12]:
df = pd.merge(df1,city_info,how='left',on=['yellow pages city','yellow pages state'])

In [13]:
df

,industry,store qty type,yellow pages state,yellow pages city,merchant_qty,store_qty,location,latitude,longitude
0,Clothing Stores,1,AK,Anchorage,64,64,"Anchorage, Anchorage, USA",61.216313,-149.894852
1,Clothing Stores,1,AK,Eagle River,4,4,"Eagle River, Eagle River, USA",47.407605,-88.295644
2,Clothing Stores,1,AK,Wasilla,1,1,"Wasilla, Wasilla, USA",61.580516,-149.440540
3,Clothing Stores,1,AL,Alabaster,7,7,"Alabaster, Alabaster, USA",33.225299,-86.803720
4,Clothing Stores,1,AL,Athens,1,1,"Athens, Athens, USA",39.328924,-82.101248
...,...,...,...,...,...,...,...,...,...
44491,Vape Shops,6~10,WI,Mount Pleasant,1,1,"Mount Pleasant, Mount Pleasant, USA",41.095433,-73.793723
44492,Vape Shops,6~10,WI,Waukesha,1,1,"Waukesha, Waukesha, USA",43.011678,-88.231481
44493,Vape Shops,6~10,WI,West Allis,1,1,"West Allis, West Allis, USA",43.015220,-88.047355
44494,Vape Shops,6~10,WV,Charleston,1,1,"Charleston, Charleston, USA",32.788436,-79.939931


In [14]:
# # 筛选数据
# selected_industry = 'Clothing Stores'  # 示例筛选条件，可以修改为用户输入
# selected_store_qty_type = '1'  # 示例筛选条件，可以修改为用户输入

# filtered_df = df[(df['industry'] == selected_industry) & 
#                  (df['store qty type'] == selected_store_qty_type)]

# # 检查筛选后的数据
# print(filtered_df.head())

# # 聚合数据，计算每个州的商家数量和店铺数量
# grouped_df = filtered_df.groupby(['yellow pages state']).agg({
#     'merchant_qty': 'sum',
#     'store_qty': 'sum'
# }).reset_index()

# # 检查聚合后的数据
# print(grouped_df.head())

# # 绘制区域热力图
# fig = px.choropleth(grouped_df, 
#                     locations='yellow pages state', 
#                     locationmode='USA-states', 
#                     color='merchant_qty', 
#                     hover_name='yellow pages state', 
#                     color_continuous_scale='Viridis',
#                     scope='usa')

# # 更新布局
# fig.update_layout(title_text='美国各州商家数量热力图')

# # 保存并展示图表
# fig.write_html("state_heatmap.html")
# fig.show()

# print("Heatmap generated and saved as state_heatmap.html")

In [15]:
# 获取唯一的行业和店铺数量类型列表
industries = df['industry'].unique()
store_qty_types = df['store qty type'].unique()

# 对店铺数量类型进行排序，并添加 "All" 选项
store_qty_types_sorted = sorted(store_qty_types, key=lambda x: (int(x.split('~')[0]) if '~' in x else (int(x[:-1]) if x.endswith('+') else int(x))))
store_qty_types_sorted.insert(0, 'All')

# 创建 Dash 应用程序
app = Dash(__name__)

app.layout = html.Div([
    html.H1("美国各州商家数量热力图"),
    html.Div([
        html.Label("选择行业:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='industry-dropdown',
            options=[{'label': industry, 'value': industry} for industry in industries],
            value=industries[0],
            style={'width': '200px'}
        ),
        html.Label("选择店铺数量类型:", style={'margin-left': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='store-qty-type-dropdown',
            options=[{'label': qty_type, 'value': qty_type} for qty_type in store_qty_types_sorted],
            value='All',
            style={'width': '200px'}
        )
    ], style={'display': 'flex', 'align-items': 'center'}),
    dcc.Graph(id='heatmap', style={'height': '80vh'})  # 调整图表高度
])

@app.callback(
    Output('heatmap', 'figure'),
    [Input('industry-dropdown', 'value'),
     Input('store-qty-type-dropdown', 'value')]
)
def update_heatmap(selected_industry, selected_store_qty_type):
    # 筛选数据
    if selected_store_qty_type == 'All':
        filtered_df = df[df['industry'] == selected_industry]
    else:
        filtered_df = df[(df['industry'] == selected_industry) & 
                         (df['store qty type'] == selected_store_qty_type)]

    # 聚合数据，计算每个州的商家数量和店铺数量
    grouped_df = filtered_df.groupby(['yellow pages state']).agg({
        'store_qty': 'sum'
    }).reset_index()

    # 生成颜色条，颜色根据店铺数量来区分
    colorscale = [[0, 'blue'], [0.5, 'yellow'], [1, 'red']]

    # 绘制统一颜色的地图
    fig = go.Figure()

    fig.add_trace(go.Choropleth(
        locations=grouped_df['yellow pages state'],
        z=[1] * len(grouped_df),  # 所有州的颜色值相同
        locationmode='USA-states',
        colorscale=[[0, 'rgb(44, 62, 80)'], [1, 'rgb(44, 62, 80)']],  # 统一深蓝色
        showscale=False,
        hoverinfo='none',  # 禁用悬停提示
        showlegend=False  # 禁用图例
    ))

    # 添加气泡标记和标签
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        locations=grouped_df['yellow pages state'],
        text=grouped_df.apply(lambda row: f"{row['yellow pages state']}<br>{row['store_qty']}", axis=1),
        mode='markers+text',
        marker=dict(
            size=10,
            color=grouped_df['store_qty'],
            colorscale=colorscale,
            colorbar=dict(title="店铺数量")
        ),
        textfont=dict(
            size=12,
            color='white'
        ),
        textposition="bottom center",
        hovertemplate='<b>%{text}</b><extra></extra>',  # 自定义提示模板
        showlegend=False  # 禁用图例
        # hoverinfo='none'  # 禁用悬停提示
    ))

    # 更新布局
    fig.update_layout(
        title_text='美国各州店铺数量热力图',
        geo=dict(
            scope='usa',
            projection=go.layout.geo.Projection(type='albers usa'),
            showlakes=True,
            lakecolor='rgb(255, 255, 255)'
        )
    )

    return fig

if __name__ == '__main__':
    # app.run_server(debug=False, host='0.0.0.0', port=8051)
    app.run_server(debug=False)